In [65]:
import pandas as pd

In [66]:
df = pd.read_csv("hotel_bookings.csv")

In [67]:
pd.options.display.max_columns = 40

# Intro

I will impute in preprocessing pipeline when doing model selection so I'm not looking at Null-values here. 

Neither will i try to catch typos since the size of the dataset is to big to make that a reasonably efficient process. 

In this cleaning process I'll focus on:

- handling duplicates
- handling outliers


# df.duplicated()

In [68]:
df.duplicated().value_counts()

# 32k rows are duplicates. Quite a lot!!
# a problem here is that a duplicate is not necessarily a "true" duplicate...
# ... because there are not sufficient identifiers for the bookings to rule out "false" duplicates.
# However, it's not that likely that a duplicate is a "false" duplicate...
# ... and the information-loss for removing a "false" duplicate is not that big.
# So I'm fine with just removing all the duplicates

False    87396
True     31994
dtype: int64

In [69]:
# duplicated() defaults to not marking the first occurence of a duplicate as a duplicate
# so we can remove all rows marked as "True" 

# using ~ to get all rows that are NOT duplicates

df = df.loc[~df.duplicated()].copy()

# outliers

In [70]:
# Overview of all numeric columns. 20 in total 
print(df.select_dtypes("number").columns.shape)
df.select_dtypes("number").columns.tolist()

(20,)


['is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'agent',
 'company',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [72]:
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,87396.000000,87396.000000,87396.000000,87396.000000,87396.000000,87396.000000,87396.000000,87396.000000,87392.000000,87396.000000,87396.000000,87396.000000,87396.000000,87396.000000,75203.000000,5259.000000,87396.000000,87396.000000,87396.000000,87396.000000
mean,0.274898,79.891368,2016.210296,26.838334,15.815541,1.005263,2.625395,1.875795,0.138640,0.010824,0.039075,0.030413,0.183990,0.271603,94.138306,183.081384,0.749565,106.337246,0.084226,0.698567
std,0.446466,86.052325,0.686102,13.674572,8.835146,1.031921,2.053584,0.626500,0.455881,0.113597,0.193775,0.369145,1.731894,0.727245,113.188172,130.557608,10.015731,55.013953,0.281533,0.831946
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,11.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,47.000000,0.000000,72.000000,0.000000,0.000000
50%,0.000000,49.000000,2016.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,169.000000,0.000000,98.100000,0.000000,0.000000
75%,1.000000,125.000000,2017.000000,37.000000,23.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,263.000000,0.000000,134.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [73]:
# I'm suspicious of max-value for adults. Each row should represent 1 booking and 1 room...
# ... and a room for 55 adults does not seem reasonable
# same goes for max-value for children-column
# same goes for max-value for babies-column

# Let's look into this!

# looking into max-values for adults and children

## adults

In [74]:
df.adults.nlargest(20)
# all values above 6 are suspicious
# perhaps even all values above 4

2173    55
1643    50
1539    40
1917    27
1962    27
1587    26
1752    26
1884    26
2003    26
2164    26
2228    20
2418    20
2417    10
2229     6
2231     5
2419     5
125      4
354      4
1023     4
6116     4
Name: adults, dtype: int64

In [75]:
# looking at value counts
df.adults.value_counts()
# I'm making a judgement here that 0-4 are the only valid numbers of adults. 
# None of the other values are reoccuring often enough for it to be reasonable that such booking alternatives are normal

2     64497
1     16503
3      5935
0       385
4        60
26        5
27        2
20        2
5         2
55        1
50        1
40        1
10        1
6         1
Name: adults, dtype: int64

In [76]:
# only looking at bookings with more than 4 adults
df.loc[df.adults > 4]
# all have been canceled

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
1539,Resort Hotel,1,304,2015,September,36,3,0,3,40,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1587,Resort Hotel,1,333,2015,September,36,5,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1643,Resort Hotel,1,336,2015,September,37,7,1,2,50,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-18
1752,Resort Hotel,1,340,2015,September,37,12,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1884,Resort Hotel,1,347,2015,September,38,19,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1917,Resort Hotel,1,349,2015,September,39,21,1,3,27,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1962,Resort Hotel,1,352,2015,September,39,24,1,3,27,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2003,Resort Hotel,1,354,2015,September,39,26,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2164,Resort Hotel,1,361,2015,October,40,3,2,5,26,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2173,Resort Hotel,1,338,2015,October,41,4,2,0,55,0.0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02


In [77]:
# I want to drop all rows where number of adults is more than 4. 
# locating index for rows where adults > 4 and dropping inplace from df
df.drop(df.loc[df.adults > 4].index, inplace = True)

## children

In [78]:
df.children.nlargest(20)

328      10.0
6748      3.0
7666      3.0
16360     3.0
18745     3.0
18746     3.0
19718     3.0
21005     3.0
22636     3.0
23407     3.0
24409     3.0
25389     3.0
25824     3.0
26613     3.0
27246     3.0
31229     3.0
37894     3.0
39265     3.0
40927     3.0
40984     3.0
Name: children, dtype: float64

In [79]:
df.children.value_counts()
# my suspiciion seems justified
# ill drop the single row with 10 children 

0.0     79012
1.0      4695
2.0      3593
3.0        75
10.0        1
Name: children, dtype: int64

In [80]:
# 328 is the index which we got from nlargest 2 cells above this one
df.drop(328, inplace = True)

## Babies

In [81]:
df.babies.nlargest(10)

46619    10
78656     9
264       2
6719      2
7896      2
14556     2
16677     2
29906     2
33332     2
37302     2
Name: babies, dtype: int64

In [82]:
# dropping rows with 9 and 10 babies
df.drop([46619, 78656], inplace = True)

# continuing with outliers

So far we have looked at children, adults and babies

Now, let's continue our investigation!

In [83]:
df.describe()
# previous_cancellations and previous_bookings_not_canceled have suspicious max-values

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87373.000000,87377.000000,87377.000000,87377.000000,87377.00000,87377.000000,75195.000000,5259.000000,87377.000000,87377.000000,87377.000000,87377.000000
mean,0.274763,79.845760,2016.210547,26.836101,15.816542,1.005104,2.625062,1.871625,0.138555,0.010609,0.039084,0.030420,0.18403,0.271605,94.140448,183.081384,0.749728,106.356792,0.084244,0.698696
std,0.446398,85.990379,0.685956,13.674631,8.835284,1.031901,2.053453,0.510567,0.454704,0.104116,0.193795,0.369185,1.73208,0.727280,113.193330,130.557608,10.016814,55.000973,0.281561,0.831978
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,11.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,9.000000,47.000000,0.000000,72.000000,0.000000,0.000000
50%,0.000000,49.000000,2016.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,14.000000,169.000000,0.000000,98.100000,0.000000,0.000000
75%,1.000000,125.000000,2017.000000,37.000000,23.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,240.000000,263.000000,0.000000,134.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,4.000000,3.000000,2.000000,1.000000,26.000000,72.00000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


# previous_cancellations & previous_bookings_not_canceled

## previous_cancellations

In [ ]:
# Noticing that index-values are following each other so could be same person abusing the booking-system or...
# ...some error causing issues with the booking (or something along these lines)

In [85]:
df.previous_cancellations.nlargest(10)

14779    26
14829    25
14838    25
14972    24
15150    24
73714    21
14997    19
14885    14
77339    13
77340    13
Name: previous_cancellations, dtype: int64

In [86]:
df.loc[df.previous_cancellations > 20]
# hard to tell anything special from just looking at these bookings
# I'll leave it as it is

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
14779,Resort Hotel,1,275,2015,October,41,4,2,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,26,0,A,A,0,Non Refund,208.0,NaN,0,Transient,50.00,0,0,Canceled,2015-01-30
14829,Resort Hotel,1,222,2015,September,38,15,1,5,2,0.0,0,FB,PRT,Groups,Corporate,0,25,0,A,A,0,Non Refund,252.0,NaN,0,Transient,49.95,0,0,Canceled,2015-03-03
14838,Resort Hotel,1,222,2015,September,38,15,1,5,2,0.0,0,FB,PRT,Groups,Corporate,0,25,0,A,A,0,Non Refund,252.0,NaN,0,Transient,38.00,0,0,Canceled,2015-03-03
14972,Resort Hotel,1,166,2015,July,29,15,0,2,2,0.0,0,FB,PRT,Groups,TA/TO,0,24,0,A,A,0,Non Refund,NaN,NaN,0,Transient,121.50,0,0,Canceled,2015-04-28
15150,Resort Hotel,1,244,2015,September,36,2,2,5,2,0.0,0,BB,IRL,Groups,TA/TO,0,24,0,A,A,0,No Deposit,96.0,NaN,0,Transient-Party,8.00,0,0,Canceled,2015-06-17
73714,City Hotel,1,265,2015,July,28,9,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,1,21,0,A,A,0,No Deposit,1.0,NaN,0,Transient-Party,62.80,0,0,Canceled,2014-10-17


## Previous_bookings_not_canceled

In [91]:
df.previous_bookings_not_canceled.nlargest(10)
# Noticing that index-values are following each other so could be same person abusing the booking-system or...
# ...some error causing issues with the booking (or something along those lines)

76731    72
76730    71
76729    70
76728    69
76727    68
76726    67
76725    66
76724    65
76723    64
76722    63
Name: previous_bookings_not_canceled, dtype: int64

In [95]:
df.previous_bookings_not_canceled.nlargest(50)
# from 72-33 it's a index-value increasing incrementally so that should be the same person

76731    72
76730    71
76729    70
76728    69
76727    68
76726    67
76725    66
76724    65
76723    64
76722    63
76721    62
76720    61
76719    60
76718    59
76716    58
76717    58
76715    57
76714    56
76713    55
76712    54
76711    53
76710    52
76709    51
76708    50
76707    49
76705    48
76706    48
76704    47
76703    46
76702    45
76700    44
76701    44
76699    43
76698    42
76697    41
76696    40
76695    39
76694    38
76693    37
76692    36
76691    35
76690    34
76689    33
74728    32
76688    32
74727    31
76687    31
14029    30
74726    30
76686    30
Name: previous_bookings_not_canceled, dtype: int64

In [97]:
df.loc[df.previous_bookings_not_canceled > 32]

# The index-values are phishy but other than that the values seem reasonable.
# Could be that it's from the same booking-company that are making batch-bookings. Or when retrieving the information...
# .. it's somehow batched and the bookings from one company are ordered

# Anyhow I will leave this as it is. Had it been a real world-setting this should have been investigated further. 

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
76689,City Hotel,0,7,2016,May,22,23,1,2,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,33,A,A,1,No Deposit,NaN,40.0,0,Transient,67.0,1,0,Check-Out,2016-05-26
76690,City Hotel,0,3,2016,May,23,30,1,2,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,34,A,A,0,No Deposit,NaN,40.0,0,Transient,65.0,0,0,Check-Out,2016-06-02
76691,City Hotel,0,13,2016,June,25,14,0,3,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,35,A,A,2,No Deposit,NaN,40.0,0,Transient,65.0,0,1,Check-Out,2016-06-17
76692,City Hotel,0,4,2016,June,26,21,0,1,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,36,A,A,1,No Deposit,NaN,40.0,0,Transient,67.0,1,0,Check-Out,2016-06-22
76693,City Hotel,0,2,2016,July,29,10,2,1,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,37,A,A,1,No Deposit,NaN,40.0,0,Transient,65.0,0,0,Check-Out,2016-07-13
76694,City Hotel,0,5,2016,July,30,18,1,1,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,38,A,A,1,No Deposit,NaN,40.0,0,Transient,65.0,0,2,Check-Out,2016-07-20
76695,City Hotel,0,2,2016,July,31,24,2,0,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,39,A,A,2,No Deposit,NaN,40.0,0,Transient-Party,65.0,0,0,Check-Out,2016-07-26
76696,City Hotel,0,1,2016,July,32,31,2,1,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,40,A,A,1,No Deposit,NaN,40.0,0,Transient,65.0,0,2,Check-Out,2016-08-03
76697,City Hotel,0,2,2016,August,33,7,2,1,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,41,A,A,0,No Deposit,NaN,40.0,0,Transient,65.0,0,0,Check-Out,2016-08-10
76698,City Hotel,0,0,2016,August,36,29,1,2,1,0.0,0,BB,PRT,Corporate,Corporate,1,3,42,A,B,0,No Deposit,NaN,40.0,0,Transient,65.0,0,0,Check-Out,2016-09-01


# continuing with outliers

In [99]:
df.describe()
# min and max values for adr are suspicious. 

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87377.000000,87373.000000,87377.000000,87377.000000,87377.000000,87377.00000,87377.000000,75195.000000,5259.000000,87377.000000,87377.000000,87377.000000,87377.000000
mean,0.274763,79.845760,2016.210547,26.836101,15.816542,1.005104,2.625062,1.871625,0.138555,0.010609,0.039084,0.030420,0.18403,0.271605,94.140448,183.081384,0.749728,106.356792,0.084244,0.698696
std,0.446398,85.990379,0.685956,13.674631,8.835284,1.031901,2.053453,0.510567,0.454704,0.104116,0.193795,0.369185,1.73208,0.727280,113.193330,130.557608,10.016814,55.000973,0.281561,0.831978
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,11.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,9.000000,47.000000,0.000000,72.000000,0.000000,0.000000
50%,0.000000,49.000000,2016.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,14.000000,169.000000,0.000000,98.100000,0.000000,0.000000
75%,1.000000,125.000000,2017.000000,37.000000,23.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,240.000000,263.000000,0.000000,134.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,4.000000,3.000000,2.000000,1.000000,26.000000,72.00000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


## adr min

In [101]:
df.adr.nsmallest(20)
# There could be bookings that are complimentary but they definetely shouldn't have negative values
# For eventual complimentary bookings its still feels reasonable to remove them as they are not "normal" data...
# ... but I will investigate those further before making a decision

14969   -6.38
0        0.00
1        0.00
125      0.00
167      0.00
168      0.00
196      0.00
197      0.00
421      0.00
428      0.00
459      0.00
568      0.00
569      0.00
618      0.00
619      0.00
665      0.00
722      0.00
774      0.00
814      0.00
815      0.00
Name: adr, dtype: float64

In [104]:
df.drop(14969,inplace = True)

In [106]:
df.loc[df.adr == 0].is_canceled.value_counts()
# very few of these bookings have been canceled
# They differ significantly from the rest of the dataset...
# ... and they are not relevant for the focus of the business use case that I am pursuing.
# I don't want the model to be affected by these rows/bookings.
# With all this said, I will remove these rows

0    1593
1     169
Name: is_canceled, dtype: int64

In [108]:
# dropping rows where adr == 0
df.drop(df.loc[df.adr == 0].index, inplace = True)

In [111]:
df.adr.nsmallest(20)
# still lots of booking with very low adr
# I need to set a limit for what constitutes a reasonable price for one day. 
# 30 seems like a reasonable start without going into detail

35059     0.26
47427     0.50
47466     1.00
47642     1.00
82659     1.00
82668     1.00
82679     1.00
83068     1.00
83203     1.00
83282     1.00
83688     1.00
83697     1.00
84556     1.00
85120     1.00
92962     1.00
104145    1.29
62011     1.48
28979     1.56
39825     1.56
82302     1.60
Name: adr, dtype: float64

In [116]:
df.loc[df.adr <= 30].is_canceled.value_counts(normalize = True)
# still significant difference from entire dataset
# dropping these rows

0    0.834477
1    0.165523
Name: is_canceled, dtype: float64

In [120]:
# dropping rows where adr is equal to or below 30
df.drop(df.loc[df.adr <= 30].index, inplace = True)

In [122]:
# New limit set to 50 for adr
df.loc[df.adr < 51].is_canceled.value_counts(normalize = True)
# signifantly lower compared to whole dataset
# hard to decide without context if these are "normal" bookings. 
# Without going into detail i'll say that these are not "normal" and thus remove them

0    0.840788
1    0.159212
Name: is_canceled, dtype: float64

In [124]:
# dropping rows with adr < 51
df.drop(df.loc[df.adr < 51].index, inplace = True)

In [134]:
# setting 70 as new limit

# is_canceled rate for less than 71 adr
print("< 71:\n",df.loc[df.adr < 71].is_canceled.value_counts(normalize = True))
print("")

# is_canceled rate for more than 70 adr
print("> 70:\n", df.loc[df.adr > 70].is_canceled.value_counts(normalize = True))

< 71:
 0    0.800363
1    0.199637
Name: is_canceled, dtype: float64

> 70:
 0    0.694216
1    0.305784
Name: is_canceled, dtype: float64


In [135]:
# Significant difference. 
# Still I will leave these values since i find 50 to be a reasonably adr compared to mean (~108).
# Somewhat arbitrary decision but I don't have enough info to make a qualified decision

## adr max

In [136]:
df.adr.nlargest()
# 5400 is not reasonably

48515     5400.0
111403     510.0
15083      508.0
103912     451.5
13142      450.0
Name: adr, dtype: float64

In [137]:
# dropping row with 5400 adr
df.drop(48515, inplace = True)

In [142]:
# looking at bookings with adr > 300
df.loc[df.adr > 300]
# very few

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
3712,Resort Hotel,0,52,2015,December,53,31,0,1,2,0.0,0,HB,PRT,Direct,Direct,0,0,0,E,F,1,No Deposit,NaN,NaN,0,Transient,332.00,1,1,Check-Out,2016-01-01
7419,Resort Hotel,1,13,2016,August,33,8,1,3,3,1.0,0,HB,ESP,Online TA,TA/TO,0,0,0,H,H,0,No Deposit,240.0,NaN,0,Transient,369.00,0,0,Canceled,2016-07-27
7543,Resort Hotel,1,105,2016,August,33,13,4,6,2,2.0,0,HB,MAR,Online TA,TA/TO,0,0,0,G,G,0,No Deposit,240.0,NaN,0,Transient,309.00,0,0,Canceled,2016-07-30
7606,Resort Hotel,1,75,2016,August,34,16,0,4,3,1.0,0,HB,ESP,Online TA,TA/TO,0,0,0,H,H,0,No Deposit,240.0,NaN,0,Transient,314.50,0,0,Canceled,2016-06-02
7673,Resort Hotel,1,26,2016,August,34,20,1,1,3,1.0,0,BB,USA,Online TA,TA/TO,0,0,0,H,H,0,No Deposit,240.0,NaN,0,Transient,304.00,0,0,Canceled,2016-07-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117125,City Hotel,0,47,2017,July,30,28,0,2,2,1.0,0,HB,SWE,Online TA,TA/TO,0,0,0,F,F,1,No Deposit,9.0,NaN,0,Transient,307.00,0,0,Check-Out,2017-07-30
117792,City Hotel,0,64,2017,August,31,4,2,4,2,2.0,0,BB,BEL,Online TA,TA/TO,0,0,0,G,G,0,No Deposit,8.0,NaN,0,Transient,305.00,0,1,Check-Out,2017-08-10
118241,City Hotel,0,190,2017,August,32,12,2,3,2,3.0,0,BB,PRT,Online TA,TA/TO,0,0,0,G,G,0,No Deposit,8.0,NaN,0,Transient,326.50,0,2,Check-Out,2017-08-17
118252,City Hotel,0,190,2017,August,32,12,2,3,2,3.0,0,BB,NOR,Online TA,TA/TO,0,0,0,G,G,0,No Deposit,8.0,NaN,0,Transient,326.50,0,2,Check-Out,2017-08-17


In [144]:
# looking at mean adr for each room type
df.groupby("reserved_room_type").adr.mean()
# Big difference between rooms but mean is way below the highest values for adr (around 500)
# Still reasonable with 500 adr I think (for instance a master suite that rarely gets booked)
# So I'll leave this as it is

reserved_room_type
A    102.503483
B     96.473581
C    170.223677
D    127.199064
E    131.885365
F    174.424457
G    185.311468
H    190.963226
L    148.000000
Name: adr, dtype: float64

# continuing with outliers

In [145]:
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,77349.000000,77349.000000,77349.000000,77349.000000,77349.000000,77349.000000,77349.000000,77349.000000,77348.000000,77349.000000,77349.000000,77349.000000,77349.000000,77349.000000,68575.000000,3409.000000,77349.000000,77349.000000,77349.000000,77349.000000
mean,0.291070,83.605632,2016.232401,27.044952,15.844652,1.023775,2.656738,1.912850,0.152389,0.010679,0.026904,0.026956,0.140247,0.261956,86.106438,175.742446,0.776494,115.889450,0.083143,0.723306
std,0.454259,86.178460,0.680709,12.968323,8.849172,1.007966,1.968317,0.490505,0.474719,0.104532,0.161804,0.320613,1.662375,0.698167,109.998055,127.646195,10.263815,47.062776,0.279034,0.838239
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,51.000000,0.000000,0.000000
25%,0.000000,15.000000,2016.000000,17.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,40.000000,0.000000,80.300000,0.000000,0.000000
50%,0.000000,55.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,169.000000,0.000000,105.500000,0.000000,1.000000
75%,1.000000,130.000000,2017.000000,36.000000,24.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,233.000000,0.000000,139.500000,0.000000,1.000000
max,1.000000,709.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,4.000000,3.000000,2.000000,1.000000,24.000000,72.000000,18.000000,535.000000,543.000000,391.000000,510.000000,8.000000,5.000000


In [147]:
# I'm fine with the rest of the columns so handling of outliers is now finished

# cleaned data, write to csv

In a real world setting I would use the cleaned data for model building but for the scope of this project there is not enough time. 

I had a tight deadline and focused on the business use case.

The performance of the model was not important for the process of learning to define a business problem. In a real world setting the performance of the model and quality of data would of course be central. 

I stressed through the data-cleaning the first time and missed a lot of things. This is the second iteration of data-cleaning and I am not going to do a second iteration of model-building as I feel that the learning prospects for doing that aren't particularly exciting.

LESSON LEARNED! Do a more thorough data-cleaning in the first iteration

In [ ]:
# here I would write the cleaned df to csv 